In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls

datalab  drive


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [0]:
import keras
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

Using TensorFlow backend.


In [0]:
data_path = 'drive/projects/QuoteGen/data'
if not os.path.exists(data_path):
  os.mkdir(data_path)

In [0]:
models_path = 'drive/projects/QuoteGen/models'
if not os.path.exists(models_path):
  os.mkdir(models_path)

In [0]:
data_txt = os.path.join(data_path, 'quotes.txt')
data_csv = os.path.join(data_path, 'quotes_all.csv')

In [0]:
quotes_ds = []
if os.path.exists(os.path.join(data_path,"quotes_pickle.pkl")):
  with open(os.path.join(data_path,"quotes_pickle.pkl"),'rb') as f:
    quotes_ds = pickle.load(f)
else:
  with open(data_csv) as f:
    for line in f.readlines():
      quotes_ds.append(line.split(";")[0])
  # Pickling the dataset
  if not os.path.exists(os.path.join(data_path,'quotes_pickle.pkl')):
    out = open(os.path.join(data_path,"quotes_pickle.pkl"),'wb')
    pickle.dump(quotes_ds, out)
print("Found %d quotes"%len(quotes_ds))

Found 75966 quotes


In [0]:
quotes_ds[0]

"Age is an issue of mind over matter. If you don't mind, it doesn't matter."

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [0]:
nb_words = 500
max_seq_len = 30

In [0]:
tokenizer = Tokenizer(num_words = nb_words)
tokenizer.fit_on_texts(quotes_ds)
sequences = tokenizer.texts_to_sequences(quotes_ds)

word_index = tokenizer.word_index
print('Found %d unique tokens.' % len(word_index))

Found 34015 unique tokens.


In [0]:
data = pad_sequences(sequences, maxlen=max_seq_len)

In [0]:
index2word = {}
for word in tokenizer.word_index:
  index2word[tokenizer.word_index.get(word)] = word

In [0]:
test = data[0]
s = ""
for temp in test:
  if not temp==0:
    s = s+ " "+ index2word[temp]
    
quotes_ds[0],s

("Age is an issue of mind over matter. If you don't mind, it doesn't matter.",
 " age is an of mind over matter if you don't mind it doesn't matter")

In [0]:
# TODO:
# Shift data to create target data

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 166,   7,  38,   4, 248, 168, 336,  30,  11,  50,
       248,  10, 264, 336], dtype=int32)